In [1]:
%load_ext tensorboard
%tensorboard --logdir ./result

In [3]:
import math
import random
from typing import List, Optional, Tuple, Union
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch import nn
import os
from torch.utils.data import IterableDataset, Dataset
import json
import numpy as np
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PretrainedConfig
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer, DefaultDataCollator, DataCollatorForTokenClassification, AutoConfig
from train_llama2_from_scratch import Config, LLM

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
AutoConfig.register("custom_gpt", Config)
AutoModelForCausalLM.register(Config, LLM)

In [22]:
# eval result
query = "北京在哪个国家？"
# query = "1+1等于几？"

reload_model = AutoModelForCausalLM.from_pretrained('./result/checkpoint-13800').to(device) 
tokenizer = AutoTokenizer.from_pretrained("./tokenizer", use_fast=True)
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(query)
input_data = {'input_ids': torch.tensor(input_ids, device=device).unsqueeze(0), "labels":None} # unsqueeze(0) to insert a dim at index 0 for batch
input_data

{'input_ids': tensor([[   2,  224, 6211,  369, 2405,  372, 1318,  681,  815]]),
 'labels': None}

In [23]:
tokenizer.decode(input_data['input_ids'][0])

'<s> 北京在哪个国家？'

In [26]:
for token in reload_model.generate(inputs=input_data, eos=tokenizer.eos_token_id, max_new_tokens=100, stream=False):
    print(tokenizer.decode(token[0]))

                        如果     如果  如果    如果  如果还有  如果  如果  如果  如果  如果   如果  如果   如果 “  如果   如果  如果  如果  如果   如果  如果   如果  如果  如果  如果 "  如果  如果  如果 


In [36]:
reload_model = AutoModelForCausalLM.from_pretrained('./result/sft/checkpoint-6441').to(device)
input = f'<s>user\n{query}</s>\n<s>assistant\n'
input_ids = tokenizer.encode(input)
input_data = {'input_ids': torch.tensor(input_ids, device=device).unsqueeze(0), "labels":None} # unsqueeze(0) to insert a dim at index 0 for batch
for token in reload_model.generate(inputs=input_data, eos=tokenizer.eos_token_id, max_new_tokens=100, stream=False):
    print(tokenizer.decode(token[0]))

 北京是首都国家，位于中国。北京；1994年，位于中国内陆，是中国内外重要的经济经济的重要组成部分；北京、上海、外交汇、外交汇处等；北京，是中国的经济经济经济发展和城市经济利益相关的重要经济体系之一。
